### Q1. Read the csv file employee.csv with the below schema as a dataframe where all columns can have null values. Print the dataframe, it's schema and write it as a parquet file.
ID (int)
Name (str)
HomeTown (str)
Salary (double)
JoiningDate (date)

In [ ]:
#Pyspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test_spark").getOrCreate()
spark

: 

In [ ]:
df = spark.read.csv('employee.csv',header = True, inferSchema=True)
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load('employee.csv')

: 

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.write.parquet('employee.parquet')
df.write.partitionBy("Name").mode("overwrite").parquet('employee.parquet')

### Q2. Read the parquet part files from the above solution as a dataframe and add a new column JoiningDate_AsString by casting the column JoiningDate as str and write it as a csv file. Print the dataframe and it's schema.

In [ ]:
df = spark.read.parquet('employee.parquet').show()

In [ ]:
spark.read.parquet('employee.parquet/*').show() # if partitions

In [ ]:
spark.read.parquet('employee.parquet/Name=Arpit').show() #for reading only one partition
#column name will not be present in the dataframe

In [ ]:
newcol_df = df.withColumn('JoiningDate_ASstring',df["JoiningDate"].cast('String'))

In [ ]:
newcol_df.show()

### Q3. Read the csv file from the above solution and compare the columns 'JoiningDate' and 'JoiningDate_AsString'. For the records which are True, add a new column 'State' and fill the values as 'Karnataka' if the 'HomeTown' is in Karnataka state, else fill the values as 'Out of State'. Print the dataframe and it's schema.

In [ ]:
#code
city_list = newcol_df.select("HomeTown").distinct().collect()
mylist = [r.HomeTown for r in city_list]
compare_df = newcol_df.withColumn("state",
                            f.when( (f.col("JoiningDate_ASstring") == f.col("JoiningDate")) 
                                   &  (f.col('HomeTown') == 'Bengaluru'), 'Karnataka').otherwise('Other state')
                        )
compare_df.show()
compare_df.printSchema()

### Q4. Create employee and department tables by reading the csv files employee.csv and department.csv respectively.
#### (i) Find the total number of employees in each department with the department's average salary.
#### (ii) Find the employees who have / had been idle for more than 30 days (who have / had not been allocated to any project) in Data Engineering department.

In [ ]:
#code
employee_df = pd.read_csv("employee.csv")
dept_df = pd.read_csv('department.csv')
employee_df


,ID,Name,HomeTown,Salary,JoiningDate
0,1,Arpit,Burhanpur,50000.0,2018-11-14
1,2,Benu,Bhubaneswar,100000.0,2018-11-19
2,3,Dilsher,Amritsar,100000.0,2018-11-19
3,4,Kiran,Bengaluru,50000.0,2018-11-15


In [ ]:
dept_df

,EmployeeID,DepartmentName,Client,OnboardedDate
0,1,Data Engineering,Funding Circle,2019-01-15
1,2,Data Engineering,Funding Circle,2018-11-19
2,3,Data Analytics,Funding Circle,2018-11-19
3,4,Data Analytics,Funding Circle,2018-12-16


### Q5. Write a function which accepts a dict and returns a validated dict. Below are the respective validations.
#### (i) Validate the type of each key and raise TypeError if validation fails
#### (ii) Validate if the key given in input dict is a new / unidentified key and raise KeyError if validation fails

    sample_input1 = {
        "job_run_id": "dummy_job_run_id",
        "job_name": "dummy_job_name",
        "impacted_records_count": 1000,
        "valid_records_count": 950,
        "invalid_records_count": 50,
        "comments": "dummy_comments"
    }

    expected_output1 = {
        "job_run_id": "dummy_job_run_id",
        "job_name": "dummy_job_name",
        "impacted_records_count": 1000,
        "valid_records_count": 950,
        "invalid_records_count": 50,
        "comments": "dummy_comments"
    }
    
    sample_input2 = {
        "job_run_id": "dummy_job_run_id",
        "job_name": "dummy_job_name",
        "impacted_records_count": 1000,
        "valid_records_count": 1000,
        "invalid_records_count": "zero",
        "comments": "dummy_comments"
    }
    
    expected_output2 = TypeError: Incorrect data type for metric invalid_records_count

    sample_input3 = {
        "job_run_id": "dummy_job_run_id",
        "job_name": "dummy_job_name",
        "impacted_records_count": 1000,
        "valid_records_count": 1000,
        "invalid_records_count": 0,
        "comments": "dummy_comments",
        "some_new_key": "some_value"
    }
    
    expected_output3 = KeyError: New / Unidentified metric some_new_key

In [ ]:
#code
sample_input1 = {
    "job_run_id": "dummy_job_run_id",
    "job_name": "dummy_job_name",
    "impacted_records_count": 1000,
    "valid_records_count": 950,
    "invalid_records_count": 50,
    "comments": "dummy_comments"
}

In [ ]:
def validate_func(key,value):
    sample_input1 = {
    "job_run_id": str,
    "job_name": str,
    "impacted_records_count": int,
    "valid_records_count": int,
    "invalid_records_count": int,
    "comments": str
    }
    
    if key not in sample_input1:
        print("KeyError: New / Unidentified metric {}".format(key))
    
    if key in sample_input1 and isinstance(value,sample_input1[key]) == False:
        print("TypeError: Incorrect data type for metric {}".format(key))
        
    
        
    

In [ ]:
sample_input2 = {
    "job_run_id": "dummy_job_run_id",
    "job_name": "dummy_job_name",
    "impacted_records_count": 1000,
    "valid_records_count": 1000,
    "invalid_records_count": "zero",
    "comments": "dummy_comments"
}

In [ ]:
for key,value in sample_input2.items():
    validate_func(key,value)

TypeError: Incorrect data type for metric invalid_records_count


In [ ]:
sample_input3 = {
    "job_run_id": "dummy_job_run_id",
    "job_name": "dummy_job_name",
    "impacted_records_count": 1000,
    "valid_records_count": 1000,
    "invalid_records_count": 0,
    "comments": "dummy_comments",
    "some_new_key": "some_value"
}


In [ ]:
for key,value in sample_input3.items():
    validate_func(key,value)

KeyError: New / Unidentified metric some_new_key


### Q6. Consider a humongous list with millions of elements. Write a function to find the cumulative sum of every 5 elements. 
#### There can be some elements which might not be of int type, the function will need to gracefully handle the exceptions and pass to the next element. Function is supposed to return a list where every element is expected to be the cumulative sum of every 5 elements in input list.

    sample_input = [50, 30, 20, 0, "catch_me", 0, "you_got_me", "did_you_really_catch", 40, 50, "20", 0, 0, 0, 10]
    expected_output = [100, 190, 200]

In [ ]:
sample_input = [50, 30, 20, 0, "catch_me", 0, "you_got_me", "did_you_really_catch", 40, 50, "20", 0, 0, 0, 10]

In [ ]:
filtered = [i for i in sample_input if isinstance(i,int) == True]

In [ ]:
filtered

[50, 30, 20, 0, 0, 40, 50, 0, 0, 0, 10]

In [ ]:
result = []
cumsum = 0
for i in range(0, len(filtered), 5):
    cumsum += sum(filtered[i:i + 5])
    result.append(cumsum)

print(result) # [100, 190, 200]

[100, 190, 200]


In [ ]:
#code
list(range(0,20,6))

[0, 6, 12, 18]

### Q7. Write a function to print second highest frequency element in the list.

    sample_input = [50, 30, 20, 0, "catch_me", 0, "you_got_me", "did_you_really_catch", 40, 50, "20", 0, 0, 0, 10]
    expected_output = 50

In [ ]:
#code
mydict = {}
for elem in sample_input:
    if elem in mydict:
        mydict[elem] +=1
    else:
        mydict[elem] = 0

In [ ]:
mydict

{50: 1,
 30: 0,
 20: 0,
 0: 4,
 'catch_me': 0,
 'you_got_me': 0,
 'did_you_really_catch': 0,
 40: 0,
 '20': 0,
 10: 0}

In [ ]:
second = sorted(list(mydict.values()))[-2]

In [ ]:
second

1

In [ ]:
for i,v in mydict.items():
    if v == second:
        print(i)

50


### Q8. Write a function which accepts a string and returns a string with the cases swaped. Do not use the built-in packages / methods like swapcase, etc to swap cases. Implement your own logic. Please try to use list comprehensions if possible.

    sample_input = "You're aweSOME !!!"
    expected_output = "yOU'RE AWEsome !!!"

In [ ]:
txt = "Hello My Name Is PETER"

x = txt.swapcase()

print(x)

hELLO mY nAME iS peter


### Q9. Write a simple function and decorate its behavior using python decorators. Also, write test cases for the function by mocking / patching the decorator's functionalitites and setting its respective return value.

In [ ]:
#code

### Q10. Write a test function for the below code snippet.

    def sample_func(a):
        result = []
        if a == 0:
            raise ValueError("0 cannot be given as an input value. Divide by zero exception.")
        else:
            for i in range(1, a+1):
                result.append([i+i, i-i, i*i, i/i])
        return result
        
#### (i) Assert the positive cases
#### (ii) Assert the negative cases
#### (iii) Assert the functions's response type
#### (iv) Assert the exceptions raised        

In [ ]:
#code